# GDELT Demo project showcase

This is the simple presentation of my findings and a bit about the skills involved in getting there. For gory details, see [README.md](https://github.com/reed9999/gdelt-demo/blob/master/README.md).

## Findings and visualizations

This project is still in early stages, but I'll post small scale findings incrementally. So far I'm just running a few simple descriptives and regressions to get a feel for various aspects of the data.  

### Goldstein scale (stability) versus media tone
**EARLY PHASE EXPLORATORY WORK**

**CAVEATS**: This isn't against the whole dataset yet and I'm guilty of fishing around for interesting regressions, although so far this is only my second.

Goldstein is a measure of propensity of each *type* of event to promote stability; avgtone is the average tone across all items (mostly media stories, I think) referencing the event. My intuition is that they should be positively correlated, because the media reflects the public's interest in stability. The most interesting story I've found so far is that the tone of coverage trends downwards for time while controlling for the Goldstein score of each kind of event:

In [1]:
## Obviously this won't run unless there are data files in the right place.
# I'm in the process of setting it up to use a miniature dataset small enough
# to reasonably place in git.
import analysis.goldsteinscale_avgtone as ga
regr = ga.GARegression()
regr.go()

#Coefficients are for date (in fraction of year) and Goldstein coefficient
#Dependent variable is average tone of documents, i.e. coverage.
regr.print_output()


/home/philip/Documents/aws-project/gdelt-demo/analysis/goldsteinscale_avgtone.py:90: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  self.prepare_data()
/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Warning (unofficial): 2 NA Goldstein of 47298229 total nulls


Coefficients on the regression: [[-0.22229802  0.13551692]]
MSRE: 7.835794230966895
r2: 0.569243224924088


```
Coefficients on the regression: [[-0.22229802  0.13551692]]
MSRE: 7.835794230966895
r2: 0.569243224924088```

Each year is associated with a -0.22 change in average tone, holding constant the theoretical propensity of the events to promote stability. Each 1.0 point increase in the Goldstein store is associated with a +0.12 change. I need to figure out the units for avgtone.

It's hard to interpret the MSRE in isolation, but it's a useful metric to compare to other regressions and other analyses. The r-squared looks pretty nice for just my first attempt at specifying a regression with two independent variables, but I should be cautious about this metric as I run more regressions and risk overfitting.

I have not yet run this on all the data, just a subset of six data files.

### Research questions
To get interesting answers, I need good questions. This is what makes this project more than just an extended SQL or PySpark tutorial. In this kind of research there's going to be some iteration in RQs, because things you think *a priori* might be interesting turn out not to be, and vice versa. So this is largely a running list of ideas, .

1. **Media tone and stability** - The Rosenstein score is a measure of how each kind of action promotes or erodes stability. The tone score is something about the tone of the media coverage of the event. See above.

1. **Actor affinity by dyad** - Do some actor dyads consistently produce higher scores (Rosenstein or media tone) than others? I'd think this would be a trivial "Yes" because relationships between allies should produce more positive news than those between adversaries. So this is exploring the obvious, but a good sanity check to make sure I'm understanding the nature of this data.

  1. **Asymmetrical relationships** - Just brainstorming. If X threatens Y more than Y threatens X that's also significant and interesting.
  
1. **Longitudinal time series questions** - This dataset is so rich that it could support a lot of this.

1. **Clustering countries or other actors** - If I want to polish up those *k*-means clustering skills, there's probably all kinds of variables I could derive ("number of threats made", "number of overtures to negotiation", "general tone of media coverage", etc.).

1. **Network graph analysis** - I don't actually know much about this topic, but clearly this data would support a lot of inferences about how actors build their networks. 



## Skills demonstrated

* **Regression analysis** (Scikit-learn LinearRegression class): First findings above. See [analysis](https://github.com/reed9999/gdelt-demo/tree/master/queries/analysis) for code. 

* **Hive QL and SQL**: Exploratory queries for now. See [queries/exploration](https://github.com/reed9999/gdelt-demo/tree/master/queries/exploration). The SQL ones should run against a local MySQL. Many of the Hive ones will run against MySQL with minimal modification.
  
* **Spark** (via PySpark): See [queries/spark-sql](https://github.com/reed9999/gdelt-demo/tree/master/queries/spark-sql).

* **AWS setup** (S3, EMR cluster creation)
  * **via CLI**: See [automation/scripts](https://github.com/reed9999/gdelt-demo/tree/master/automation/scripts).
  * **via boto3**: See [automation](https://github.com/reed9999/gdelt-demo/tree/master/automation).